In [24]:
# import libraries
import subprocess
import json
from constants import *
import os
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [25]:
# import mnemonic from env
mnemonic = os.getenv('MNEMONIC')

In [26]:
# connect to local ETH/ geth
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)


In [27]:
# create a function for derive wallet
def derive_wallets(mnemonic,coin,numderive):
    command = f'./derive -g --mnemonic="{mnemonic}" --col=path,address,privkey,pubkey,pubkeyhash, --coin="{coin}" --numderive="{numderive}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys 

In [28]:
# test the function 
derive_wallets(mnemonic,ETH,3) 

[{'path': "m/44'/60'/0'/0/0",
  'address': '0x6b5255EA1E97a2242F342a1961296f5119166346',
  'xprv': 'xprvA4JJW9fuduL1a91oz5WHmd2uS5oag3HZC22dvZJewe8LNYsmY3fEXwFCW1HeXBTET4kE6pkS4R8ni5qMWtu1zafcPzmTkHBJfesLFxW3ETJ',
  'xpub': 'xpub6HHeufCoUGtJnd6H673J8kydz7e55W1QZExEiwiGVyfKFMCv5ayV5jZgMHLGMhZAc35QrwBpUGgoyULLXzz3NpuAPtLnwCSoYJyvGQbiZQL',
  'privkey': '0xc7a5b67680278e52fa39a37d7aec7ad8cedb590d424720fcf8366f2ca2e3882d',
  'pubkey': '034d46f291221ac76ac8a675928113fca68d2b25c643029db584dcc9252286fe9c',
  'pubkeyhash': 'e3a284fa36e268232fba94dd056eef8d9875c9a1',
  'index': 0},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0xB61D6675b8639F2f9d6907E51F708d8b7eB2ff5D',
  'xprv': 'xprvA4JJW9fuduL1cNnYGcQ6Wnj5CUUxrUUga5ZdbttjFZuX2qrfU6djrzfkzqwspLK9Gg5mJa6ZMyBY4cPtVjcoV1d7KpeQWWR2i1sKzYJrEna',
  'xpub': 'xpub6HHeufCoUGtJprs1Ndw6svfokWKTFwCXwJVEQHJLouSVueBp1dwzQnzEr9pz9ixPaFuRLyvvRhUPQ1R4akEZ21xxJqE9JLq47pLfgFnFr7y',
  'privkey': '0x0e7db6b9f9623b3f7078c256677e9904b6a249f4e0b7c2a6da8a553a69273e93',

In [29]:
# Create an object
coins = { 'ETH' : derive_wallets(mnemonic,ETH, 3),
         'btc-test' : derive_wallets(mnemonic,BTCTEST, 3)}

In [30]:
# test the object
coins['btc-test'][1]

{'path': "m/44'/1'/0'/0/1",
 'address': 'mrGsVCNXohNcGn5DJdp8cFhhhm6a4cXHTL',
 'xprv': 'tprv8juakpxfxsxJso14iHqxuWxway2XeTHTzTG9hkPtEM6tMMhR2CPoFxBbrBUWnfNMCMkhNvhM9rWoUfEf6h6GU2qFKDhUfMx7cjWQUkK4LV2',
 'xpub': 'tpubDGbcuEzv7FdymG2rbwWZJvd49zYTonUNZkrvzGSBecuHBqxBebDPSSoU2MdFhySAW5yTVv9CjreRhnC3r3W363xrYddUNhcMayQK5Axo6ha',
 'privkey': 'cNfTo7QZbHxeiFUvgsyNP96kCn1YXmTMNDLaQxna57iY7a9MhQMg',
 'pubkey': '03c644babaecd33285e4c3f825b7f73366a1db94957b4e436b36946aeca81da157',
 'pubkeyhash': '75ff8bf7136ec3cda6788db052c7d3fc786f8db9',
 'index': 1}

In [31]:
# create a function that convert the privkey string in a child key to an account object.
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [57]:
# create function for raw transaction 
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
    


In [63]:
# create a function to hold Ethereum 
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH,5)[0]['privkey'])


eth
0xc7a5b67680278e52fa39a37d7aec7ad8cedb590d424720fcf8366f2ca2e3882d


In [61]:
# create a function to send txn
def send_txn(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        signed_txn = account.sign_transaction(txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)



In [62]:
#use the send_txn function to send transactions 
send_txn(ETH,eth_acc,'0xB61D6675b8639F2f9d6907E51F708d8b7eB2ff5D',687658768765876784)


'0xa166d561338021b312410565cee718d811060a1d169ca0d9b456f64d9971bd81'